## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-17-13-07-33 +0000,nypost,‘Melania’ Trump documentary trailer shows run-...,https://nypost.com/2025/12/17/us-news/melania-...
1,2025-12-17-13-06-14 +0000,bbc,Why this month's inflation figure may be good ...,https://www.bbc.com/news/articles/c0r9x1xx0x7o...
2,2025-12-17-13-05-03 +0000,wapo,"In tribute to late wife, husband rebuilds her ...",https://www.washingtonpost.com/lifestyle/2025/...
3,2025-12-17-13-04-00 +0000,wsj,TransAlta Gets Energy Department Order to Keep...,https://www.wsj.com/business/energy-oil/transa...
4,2025-12-17-13-02-38 +0000,nypost,Rob Reiner’s daughter found his body — but did...,https://nypost.com/2025/12/17/us-news/rob-rein...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2447/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,51
58,bondi,29
38,reiner,23
553,new,20
59,beach,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
299,2025-12-16-19-36-00 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...,131
185,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...,120
308,2025-12-16-19-05-48 +0000,nypost,Trump stands by chief of staff Susie Wiles aft...,https://nypost.com/2025/12/16/us-news/trump-st...,115
305,2025-12-16-19-17-01 +0000,nypost,Trump thinks Russia’s Putin wants all of Ukrai...,https://nypost.com/2025/12/16/world-news/trump...,113
396,2025-12-16-13-33-47 +0000,nypost,White House chief of staff Susie Wiles: AG Pam...,https://nypost.com/2025/12/16/us-news/ag-pam-b...,113


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
299,131,2025-12-16-19-36-00 +0000,nyt,"Susie Wiles, Trump’s Chief of Staff, Criticize...",https://www.nytimes.com/2025/12/16/us/politics...
184,94,2025-12-17-01-45-44 +0000,bbc,"What we know about Nick Reiner, son charged in...",https://www.bbc.com/news/articles/cvgj14l3v8vo...
272,77,2025-12-16-21-05-32 +0000,nypost,Australia opposition party launches new antise...,https://nypost.com/2025/12/16/world-news/austr...
8,71,2025-12-17-12-59-17 +0000,nypost,Warner Bros. Discovery board urges shareholder...,https://nypost.com/2025/12/17/business/warner-...
185,69,2025-12-17-01-32-56 +0000,nypost,Trump orders blockade of oil tankers from ‘for...,https://nypost.com/2025/12/16/us-news/trump-or...
74,57,2025-12-17-11-00-00 +0000,latimes,Lopez: Nearly every house on their west Altade...,https://www.latimes.com/california/story/2025-...
204,56,2025-12-17-00-24-00 +0000,wsj,New Video Shows Suspected Brown Gunman’s Movem...,https://www.wsj.com/us-news/brown-university-s...
216,48,2025-12-16-23-59-00 +0000,wsj,The Trump administration expanded its travel b...,https://www.wsj.com/politics/policy/trump-admi...
153,47,2025-12-17-04-00-00 +0000,wsj,"For Ukraine’s Allies in Europe, a Bad Deal Wit...",https://www.wsj.com/world/europe/for-ukraines-...
158,42,2025-12-17-03-40-00 +0000,wsj,The Senate majority leader said Congress won’t...,https://www.wsj.com/politics/policy/house-gop-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
